In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import gseapy as gp
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand

p = os.path.dirname(os.getcwd())
sys.path.append(p)

from scripts.utils import *
from scripts.plotting import *

from sm_utils import *

In [2]:
config_file = 'config.yml'
with open(config_file) as f:
    config = yaml.safe_load(f)

## 231010 process metadata

In [4]:
df = process_lr_metadata(config['lr']['meta'], ['human', 'mouse'], datasets_per_talon_run=8)

In [23]:
datasets_per_talon_run=8

In [24]:
# get number to mod each talon run by
# df['n_datasets'] = df[['dataset', 'species']].groupby('species').count()[['dataset']].transform('
# df['mod_num'] = np.ceil(df.n_datasets/datasets_per_talon_run)
# df.head()

df['n_datasets'] = df[['dataset', 'species']].groupby('species')[['dataset']].transform('count')
df['mod_num'] = np.ceil(df.n_datasets/datasets_per_talon_run)
df.head()

,ENCODE_experiment_id,dataset,sample,sample_display,general_tissue_cell_type,fig1_tissue_label,health_status,tissue_or_cell_line,sample_color_hex_code,matching_mouse_samples,...,RIN,spikeins,species,age,sex,genotype,matching_human_samples,talon_run,n_datasets,mod_num
0,ENCSR989ZYL,a673_1_1,a673,A673,a673,bone,NaN,cell_line,#de3700,NaN,...,9.8,True,human,NaN,NaN,NaN,NaN,1,138,18.0
1,ENCSR989ZYL,a673_1_2,a673,A673,a673,bone,NaN,cell_line,#de3700,NaN,...,9.8,True,human,NaN,NaN,NaN,NaN,2,138,18.0
2,ENCSR081NRO,adrenal_gland_1_1,adrenal_gland,Adrenal gland,adrenal gland,adrenal gland,NaN,tissue,#e69b00,"adrenal_gland,adrenal_10d,adrenal_14d,adrenal_...",...,9.8,False,human,NaN,NaN,NaN,NaN,3,138,18.0
3,ENCSR563RLX,adrenal_gland_2_1,adrenal_gland,Adrenal gland,adrenal gland,adrenal gland,NaN,tissue,#e69b00,"adrenal_gland,adrenal_10d,adrenal_14d,adrenal_...",...,NaN,True,human,NaN,NaN,NaN,NaN,4,138,18.0
4,ENCSR995WKW,adrenal_gland_3_1,adrenal_gland,Adrenal gland,adrenal gland,adrenal gland,NaN,tissue,#e69b00,"adrenal_gland,adrenal_10d,adrenal_14d,adrenal_...",...,9.4,False,human,NaN,NaN,NaN,NaN,5,138,18.0


In [27]:
df['talon_run'] = (df.sort_values(['species', 'dataset'],
                                ascending=[True, True])\
                                .groupby('species')\
                                .cumcount() + 1).to_numpy()\
                                % df.mod_num.to_numpy()

In [28]:
df

,ENCODE_experiment_id,dataset,sample,sample_display,general_tissue_cell_type,fig1_tissue_label,health_status,tissue_or_cell_line,sample_color_hex_code,matching_mouse_samples,...,RIN,spikeins,species,age,sex,genotype,matching_human_samples,talon_run,n_datasets,mod_num
0,ENCSR989ZYL,a673_1_1,a673,A673,a673,bone,NaN,cell_line,#de3700,NaN,...,9.8,True,human,NaN,NaN,NaN,NaN,1.0,138,18.0
1,ENCSR989ZYL,a673_1_2,a673,A673,a673,bone,NaN,cell_line,#de3700,NaN,...,9.8,True,human,NaN,NaN,NaN,NaN,2.0,138,18.0
2,ENCSR081NRO,adrenal_gland_1_1,adrenal_gland,Adrenal gland,adrenal gland,adrenal gland,NaN,tissue,#e69b00,"adrenal_gland,adrenal_10d,adrenal_14d,adrenal_...",...,9.8,False,human,NaN,NaN,NaN,NaN,3.0,138,18.0
3,ENCSR563RLX,adrenal_gland_2_1,adrenal_gland,Adrenal gland,adrenal gland,adrenal gland,NaN,tissue,#e69b00,"adrenal_gland,adrenal_10d,adrenal_14d,adrenal_...",...,NaN,True,human,NaN,NaN,NaN,NaN,4.0,138,18.0
4,ENCSR995WKW,adrenal_gland_3_1,adrenal_gland,Adrenal gland,adrenal gland,adrenal gland,NaN,tissue,#e69b00,"adrenal_gland,adrenal_10d,adrenal_14d,adrenal_...",...,9.4,False,human,NaN,NaN,NaN,NaN,5.0,138,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,ENCSR909NPF,hippocampus_2mo_f_1,hippocampus_2mo,Hippocampus PNM2,hippocampus,hippocampus,NaN,tissue,#c96579,NaN,...,9.0,True,mouse,2mo,f,b6/cast,"['brain', 'pgp1_excite_neuron', 'pgp1_astro']",11.0,127,16.0
123,ENCSR909NPF,hippocampus_2mo_f_2,hippocampus_2mo,Hippocampus PNM2,hippocampus,hippocampus,NaN,tissue,#c96579,NaN,...,9.0,True,mouse,2mo,f,b6/cast,"['brain', 'pgp1_excite_neuron', 'pgp1_astro']",12.0,127,16.0
124,ENCSR972DID,hippocampus_2mo_m_1,hippocampus_2mo,Hippocampus PNM2,hippocampus,hippocampus,NaN,tissue,#c96579,NaN,...,9.0,True,mouse,2mo,m,b6/cast,"['brain', 'pgp1_excite_neuron', 'pgp1_astro']",13.0,127,16.0
125,ENCSR972DID,hippocampus_2mo_m_2,hippocampus_2mo,Hippocampus PNM2,hippocampus,hippocampus,NaN,tissue,#c96579,NaN,...,9.0,True,mouse,2mo,m,b6/cast,"['brain', 'pgp1_excite_neuron', 'pgp1_astro']",14.0,127,16.0


In [29]:
df[['dataset', 'species', 'talon_run']].groupby(['species', 'talon_run']).count()

dataset
species talon_run         
human   0.0              7
        1.0              8
        2.0              8
        3.0              8
        4.0              8
        5.0              8
        6.0              8
        7.0              8
        8.0              8
        9.0              8
        10.0             8
        11.0             8
        12.0             8
        13.0             7
        14.0             7
        15.0             7
        16.0             7
        17.0             7
mouse   0.0              7
        1.0              8
        2.0              8
        3.0              8
        4.0              8
        5.0              8
        6.0              8
        7.0              8
        8.0              8
        9.0              8
        10.0             8
        11.0             8
        12.0             8
        13.0             8
        14.0             8
        15.0             8

In [31]:
df[['dataset', 'species', 'talon_run']].groupby(['species', 'talon_run']).count().reset_index().groupby(['speciess']).sum()

,talon_run
dataset,
7,75.0
8,198.0


In [8]:
datasets_per_talon_run = 7
138/8
138/18

7.666666666666667

In [31]:
# split up by species
# temp2 = pd.DataFrame()
# for i,s in enumerate(df.species.unique().tolist()):
#     temp = df.loc[df.species==s].copy(deep=True)
#     temp = temp.sort_values(by='dataset', ascending=True)
temp = df.copy(deep=True)
temp['talon_run_num'] = ((temp.sort_values(['species', 'dataset'], 
                                    ascending=[True, True])\
                                    .groupby('species')\
                                    .cumcount() + 1)\
                                    %datasets_per_talon_run).tolist()

# temp['talon_run_num'] = temp['num']%datasets_per_talon_run

In [30]:
temp.num.unique()

array([1, 2, 3, 4, 5, 6, 0])

In [ ]:
df['flowcell'] = df.sort_values(['genotype', 'mouse_id'],
                                ascending=[True, True])\
                                .groupby(['mouse_id']) \
                                .cumcount() + 1

In [14]:
# species=['human', 'mouse']

# # lr stuff
# for i,s in enumerate(species):
#     f = expand(config['lr']['meta'], species=s)[0]
#     if i == 0:
#         lr_df = pd.read_csv(f, sep='\t')
#         lr_df['species'] = s
#     else:
#         temp = pd.read_csv(f, sep='\t')
#         temp['species'] = s
#         lr_df = pd.concat([lr_df, temp], axis=0)
        
# # for i, f in enumerate(expand(config['lr']['meta'], species=species)):
# #     if i == 0:
# #         lr_df = pd.read_csv(f, sep='\t')
# lr_meta = process_encode_metadata(expand(config['lr']['encode_meta'], species=species)[0])


In [4]:
df.species.unique()

array(['human', 'mouse'], dtype=object)

In [6]:
df.head()

,ENCODE_experiment_id,dataset,sample,sample_display,general_tissue_cell_type,fig1_tissue_label,health_status,tissue_or_cell_line,sample_color_hex_code,matching_mouse_samples,...,document_urls,document_labels,platform,RIN,spikeins,species,age,sex,genotype,matching_human_samples
0,ENCSR989ZYL,a673_1_1,a673,A673,a673,bone,NaN,cell_line,#de3700,NaN,...,https://www.encodeproject.org/documents/6d583a...,'ENCODE Long Read RNA-Seq Analysis Protocol fo...,Pacific Biosciences Sequel II,9.8,True,human,NaN,NaN,NaN,NaN
1,ENCSR989ZYL,a673_1_2,a673,A673,a673,bone,NaN,cell_line,#de3700,NaN,...,https://www.encodeproject.org/documents/6d583a...,'ENCODE Long Read RNA-Seq Analysis Protocol fo...,Pacific Biosciences Sequel II,9.8,True,human,NaN,NaN,NaN,NaN
2,ENCSR081NRO,adrenal_gland_1_1,adrenal_gland,Adrenal gland,adrenal gland,adrenal gland,NaN,tissue,#e69b00,"adrenal_gland,adrenal_10d,adrenal_14d,adrenal_...",...,https://www.encodeproject.org/documents/54012f...,'non-size selected cDNA libraries for use in P...,Pacific Biosciences Sequel,9.8,False,human,NaN,NaN,NaN,NaN
3,ENCSR563RLX,adrenal_gland_2_1,adrenal_gland,Adrenal gland,adrenal gland,adrenal gland,NaN,tissue,#e69b00,"adrenal_gland,adrenal_10d,adrenal_14d,adrenal_...",...,https://www.encodeproject.org/documents/3baa46...,"'PacBio libraries v3 (October, 2020) Protocol ...",Pacific Biosciences Sequel II,NaN,True,human,NaN,NaN,NaN,NaN
4,ENCSR995WKW,adrenal_gland_3_1,adrenal_gland,Adrenal gland,adrenal gland,adrenal gland,NaN,tissue,#e69b00,"adrenal_gland,adrenal_10d,adrenal_14d,adrenal_...",...,https://www.encodeproject.org/documents/54012f...,'non-size selected cDNA libraries for use in P...,Pacific Biosciences Sequel,9.4,False,human,NaN,NaN,NaN,NaN


In [7]:
df.tail()

,ENCODE_experiment_id,dataset,sample,sample_display,general_tissue_cell_type,fig1_tissue_label,health_status,tissue_or_cell_line,sample_color_hex_code,matching_mouse_samples,...,document_urls,document_labels,platform,RIN,spikeins,species,age,sex,genotype,matching_human_samples
122,ENCSR909NPF,hippocampus_2mo_f_1,hippocampus_2mo,Hippocampus PNM2,hippocampus,hippocampus,NaN,tissue,#c96579,NaN,...,https://www.encodeproject.org/documents/bf543f...,'ENCODE Long Read RNA-Seq Analysis Protocol fo...,Pacific Biosciences Sequel II,9.0,True,mouse,2mo,f,b6/cast,"['brain', 'pgp1_excite_neuron', 'pgp1_astro']"
123,ENCSR909NPF,hippocampus_2mo_f_2,hippocampus_2mo,Hippocampus PNM2,hippocampus,hippocampus,NaN,tissue,#c96579,NaN,...,https://www.encodeproject.org/documents/bf543f...,'ENCODE Long Read RNA-Seq Analysis Protocol fo...,Pacific Biosciences Sequel II,9.0,True,mouse,2mo,f,b6/cast,"['brain', 'pgp1_excite_neuron', 'pgp1_astro']"
124,ENCSR972DID,hippocampus_2mo_m_1,hippocampus_2mo,Hippocampus PNM2,hippocampus,hippocampus,NaN,tissue,#c96579,NaN,...,https://www.encodeproject.org/documents/bf543f...,'ENCODE Long Read RNA-Seq Analysis Protocol fo...,Pacific Biosciences Sequel II,9.0,True,mouse,2mo,m,b6/cast,"['brain', 'pgp1_excite_neuron', 'pgp1_astro']"
125,ENCSR972DID,hippocampus_2mo_m_2,hippocampus_2mo,Hippocampus PNM2,hippocampus,hippocampus,NaN,tissue,#c96579,NaN,...,https://www.encodeproject.org/documents/bf543f...,'ENCODE Long Read RNA-Seq Analysis Protocol fo...,Pacific Biosciences Sequel II,9.0,True,mouse,2mo,m,b6/cast,"['brain', 'pgp1_excite_neuron', 'pgp1_astro']"
126,ENCSR038FOD,hippocampus_10d_f_1,hippocampus_10d,Hippocampus,hippocampus,hippocampus,NaN,tissue,NaN,NaN,...,https://www.encodeproject.org/documents/bf543f...,'ENCODE Long Read RNA-Seq Analysis Protocol fo...,Pacific Biosciences Sequel II,9.0,True,mouse,10d,f,b6/cast,"['brain', 'pgp1_excite_neuron', 'pgp1_astro']"


In [18]:
len(lr_df.loc[lr_df.species=='mouse'])

126